<a href="https://colab.research.google.com/github/JOSEPHREDDY07/DeepLearning/blob/master/tf_udemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#   import TensorFlow
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))
print("Keras version: {}".format(tf.keras.__version__))

TensorFlow version: 1.15.0
Keras version: 2.2.4-tf


In [3]:
sess = tf.Session()

# Verify we can print a string
hello = tf.constant("Hello UNP from TensorFlow")
print(sess.run(hello))

b'Hello UNP from TensorFlow'


In [4]:
#   Perform some simple math
a = tf.constant(20)
b = tf.constant(22)
print('a + b = {0}'.format(sess.run(a + b)))

a + b = 42
